# 9. Consolidate observations

## Setup

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
sample_file <- '../preprocessed_data/sample_2019-08-24.feather'
company_groups_file <- '../preprocessed_data/company_groups_2019-08-24.feather'
m_as_file <- '../preprocessed_data/m_as_2019-08-24.feather'
pipelines_2004_file <- "../preprocessed_data/pipelines_2004_selected_2019-08-26.feather"

## Read data

In [3]:
sample <- feather::read_feather(sample_file)
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
glimpse(sample)

Warning message:
“Coercing int64 to double”

Observations: 1,026
Variables: 20
$ OPERATOR_ID           <chr> "300", "300", "300", "300", "300", "300", "300"…
$ YEAR                  <dbl> 2017, 2017, 2011, 2014, 2012, 2010, 2018, 2018,…
$ NAME                  <chr> "PLAINS PIPELINE, L.P.", "PLAINS PIPELINE, L.P.…
$ COMMODITY             <chr> "hvl", "non-hvl", "non-hvl", "non-hvl", "non-hv…
$ MILES                 <dbl> 72.900, 0.000, 326.800, NA, 265.170, 62.280, 29…
$ AGE_UNKNOWN_MILES     <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000…
$ MILES_PRE_1940        <dbl> 0.00, 3.72, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000…
$ MILES_1950            <dbl> 0.000, 0.000, 0.800, 0.000, 0.800, 244.980, 0.0…
$ MILES_1960            <dbl> 0.000, 0.000, 188.430, 0.000, 188.430, 4.760, 0…
$ MILES_1970            <dbl> 0.000, 0.000, 1.170, 0.000, 1.170, 0.000, 0.000…
$ MILES_1980            <dbl> 0.000, 0.000, 4.430, 0.000, 4.430, 0.000, 0.000…
$ MILES_1990      

In [4]:
company_groups <- feather::read_feather(company_groups_file)
glimpse(company_groups)

Observations: 39
Variables: 2
$ members <chr> "31618", "30829", "3445", "22610", "12105", "39504", "15485",…
$ name    <chr> "Enterprise Products (Group)", "Enterprise Products (Group)",…


In [5]:
m_as_orig <- feather::read_feather(m_as_file)
glimpse(m_as_orig)

Observations: 8
Variables: 4
$ members    <chr> "Sunoco (Group)", "Energy Transfer (Group)", "Kinder Morga…
$ name       <chr> "Sunoco (Group)", "Sunoco (Group)", "Kinder Morgan (Group)…
$ start_year <chr> "2017", "2017", NA, NA, "2017", "2017", "2018", "2018"
$ end_year   <chr> NA, NA, "2013", "2013", NA, NA, NA, NA


In [6]:
pipelines_2004 <- feather::read_feather(pipelines_2004_file)
glimpse(pipelines_2004)

Warning message:
“Coercing int64 to double”

Observations: 2,730
Variables: 16
$ OPERATOR_ID    <chr> "31336", "4805", "8175", "26302", "32147", "4906", "19…
$ YEAR           <dbl> 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, …
$ NAME           <chr> "CHEVRON U.S.A. INC", "EXPLORER PIPELINE CO", "INTERST…
$ COMMODITY      <chr> "CRUDE OIL", "PETROLEUM & REFINED PRODUCTS", "PETROLEU…
$ MILES          <dbl> 6.000, 1204.110, 8.000, 13.000, 1629.158, 373.000, 40.…
$ MILES_PRE_1940 <dbl> 0.000, 0.000, 0.000, 0.000, 26.841, 1.000, 0.000, 0.00…
$ MILES_1940     <dbl> 0.000, 0.000, 0.000, 0.000, 119.533, 101.000, 0.000, 0…
$ MILES_1950     <dbl> 0.000, 7.200, 0.000, 0.000, 585.263, 126.000, 0.000, 0…
$ MILES_1960     <dbl> 10.647, 7.800, 11.600, 0.000, 107.873, 342.000, 0.000,…
$ MILES_1970     <dbl> 0.000, 593.200, 0.000, 0.000, 243.374, 43.000, 0.000, …
$ MILES_1980     <dbl> 0.000, 18.300, 0.000, 89.000, 91.639, 45.000, 0.000, 0…
$ MILES_1990     <dbl> 0.000, 20.800, 0.000, 0.000, 110.908, 42.000, 0.000, 0…
$ MILES_2000     <

## 9.1 Add two years of data for crude

Adding the data for crude the years 2008/2009 offers us a relatively easy way to expand our sample. We cannot go any further, as definition of incidents was change, 2008/2009 can still be used to calculate the three years average that we use to calculate the impact of change on the incidents rates for the observation 2010-2012.

In [7]:
crude_2008 <- subset(pipelines_2004, YEAR >= 2008 & COMMODITY == "CRUDE OIL")
crude_2008$COMMODITY <- "crude"
glimpse(crude_2008)

Observations: 301
Variables: 16
$ OPERATOR_ID    <chr> "31336", "19585", "31552", "11733", "31937", "18667", …
$ YEAR           <dbl> 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, …
$ NAME           <chr> "CHEVRON U.S.A. INC", "KINDER MORGAN CANADA INC.", "AL…
$ COMMODITY      <chr> "crude", "crude", "crude", "crude", "crude", "crude", …
$ MILES          <dbl> 6.000, 40.000, 34.630, 35.250, 2.000, 37.150, 4.600, 4…
$ MILES_PRE_1940 <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 4.340, 0.000, 0.000…
$ MILES_1940     <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000…
$ MILES_1950     <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000…
$ MILES_1960     <dbl> 10.647, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.00…
$ MILES_1970     <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 4.600, 0.000…
$ MILES_1980     <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 22.070, 0.000, 28.0…
$ MILES_1990     <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 65.00…
$ MILES_2000     <db

In [8]:
crude_2008 %>%
    filter(OPERATOR_ID %in% sample$OPERATOR_ID) %>%
    mutate(PERC_OFFSHORE = round(PERC_OFFSHORE, 2)) %>%
    {table(.$PERC_OFFSHORE)}


   0 0.02 0.03 0.05 0.06 0.12 0.22 0.28 
  62    1    2    2    2    1    1    1 

In [9]:
crude_2008 %>%
    filter(OPERATOR_ID %in% sample$OPERATOR_ID) %>%
    filter(PERC_OFFSHORE >= 0.01) %>%
    select(OPERATOR_ID, YEAR, NAME, COMMODITY, MILES, AVG_AGE, PERC_OFFSHORE)

OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AVG_AGE,PERC_OFFSHORE
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
300,2008,"PLAINS PIPELINE, L.P.",crude,997.300,58.39192,0.05352059
4906,2008,EXXONMOBIL PIPELINE CO,crude,918.000,52.06991,0.06134342
31174,2008,"SHELL PIPELINE CO., L.P.",crude,1269.000,41.59168,0.02981423
2731,2008,CHEVRON PIPE LINE CO,crude,397.200,70.20813,0.21517887
31174,2009,"SHELL PIPELINE CO., L.P.",crude,1264.000,41.59168,0.02843748
2731,2009,CHEVRON PIPE LINE CO,crude,287.920,73.63542,0.11625909
4906,2009,EXXONMOBIL PIPELINE CO,crude,900.000,50.03819,0.06249349
31189,2009,BP PIPELINE (NORTH AMERICA) INC.,crude,476.000,60.18519,0.01652551
31618,2009,ENTERPRISE PRODUCTS OPERATING LLC,crude,60.142,17.59743,0.28236345


In [10]:
operators_w_offshore <- crude_2008 %>%
    filter(OPERATOR_ID %in% sample$OPERATOR_ID) %>%
    filter(PERC_OFFSHORE >= 0.01) %>%
    select(OPERATOR_ID)
operators_w_offshore

OPERATOR_ID
<chr>
300
4906
31174
2731
31174
2731
4906
31189
31618


In [11]:
subset(sample, OPERATOR_ID %in% operators_w_offshore & COMMODITY == 'crude')

OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,PERC_OFFSHORE,AVG_AGE,PARENT,INCIDENTS,SIGNIFICANT_INCIDENTS
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>


All the operators seem to have either sold off their crude segment, or the new regulations have changed something about reporting.

### 9.1.1 Consolidate observations

Before 2010, business units were sometimes reported separately, within the same OPERATOR ID. We consolidate those at the OPERATOR ID level.

In [12]:
crude_2008 <- crude_2008 %>%
    group_by(OPERATOR_ID, YEAR) %>%
    summarize(NAME = first(NAME),
              COMMODITY = first(COMMODITY),
              MILES = sum(MILES), 
              MILES_PRE_1940 = sum(MILES_PRE_1940), 
              MILES_1940 = sum(MILES_1940), 
              MILES_1950 = sum(MILES_1950), 
              MILES_1960 = sum(MILES_1960), 
              MILES_1970 = sum(MILES_1970), 
              MILES_1980 = sum(MILES_1980), 
              MILES_1990 = sum(MILES_1990), 
              MILES_2000 = sum(MILES_2000), 
              MILES_2010 = sum(MILES_2010)) %>%
    ungroup()

In [13]:
crude_2008$AVG_AGE = (crude_2008$MILES_PRE_1940 * 90 +
                      crude_2008$MILES_1940 * 75 +
                      crude_2008$MILES_1950 * 65 +
                      crude_2008$MILES_1960 * 55 +
                      crude_2008$MILES_1970 * 45 +
                      crude_2008$MILES_1980 * 35 +
                      crude_2008$MILES_1990 * 25 +
                      crude_2008$MILES_2000 * 15 +
                      crude_2008$MILES_2010 * 5) / 
                     (crude_2008$MILES_PRE_1940 + crude_2008$MILES_1940 + crude_2008$MILES_1950 + crude_2008$MILES_1960 + 
                      crude_2008$MILES_1970 + crude_2008$MILES_1980 + crude_2008$MILES_1990 + crude_2008$MILES_2000 + 
                      crude_2008$MILES_2010)

### 9.1.2 Merge

In [14]:
colnames(sample) [!(colnames(sample) %in% colnames(crude_2008))]

[1] "AGE_UNKNOWN_MILES"     "PERC_OFFSHORE"         "PARENT"               
[4] "INCIDENTS"             "SIGNIFICANT_INCIDENTS"

In [15]:
crude_2008$AGE_UNKNOWN_MILES <- NA
crude_2008$PERC_OFFSHORE <- NA
crude_2008$PARENT <- NA
crude_2008$INCIDENTS <- NA
crude_2008$SIGNIFICANT_INCIDENTS <- NA

In [16]:
sample <- rbind(sample, crude_2008)

## 9.2 Calculate change

In [17]:
sample <- sample %>%
    group_by(OPERATOR_ID, COMMODITY) %>%
    arrange(YEAR) %>%
    mutate(CHANGE = MILES - lag(MILES, 1))
glimpse(subset(sample, YEAR > 2010))

Observations: 920
Variables: 21
Groups: OPERATOR_ID, COMMODITY [132]
$ OPERATOR_ID           <chr> "300", "395", "1845", "1845", "2170", "2552", "…
$ YEAR                  <dbl> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011,…
$ NAME                  <chr> "PLAINS PIPELINE, L.P.", "AMOCO OIL CO", "BUCKE…
$ COMMODITY             <chr> "non-hvl", "non-hvl", "hvl", "non-hvl", "non-hv…
$ MILES                 <dbl> 326.800, 527.000, 47.644, 4043.393, 310.700, 40…
$ AGE_UNKNOWN_MILES     <dbl> 0.000, 0.000, 0.000, 260.106, 0.000, 14.000, 0.…
$ MILES_PRE_1940        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ MILES_1940            <dbl> 0.00, 15.00, 0.00, 801.90, 0.00, 0.00, 0.00, 0.…
$ MILES_1950            <dbl> 0.800, 0.000, 0.000, 1253.159, 45.530, 0.100, 0…
$ MILES_1960            <dbl> 188.430, 1.000, 0.606, 1764.684, 188.000, 1156.…
$ MILES_1970            <dbl> 1.170, 2.000, 22.796, 398.240, 23.100, 853.200,…
$ MILES_1980            <dbl> 4.430, 17.000, 0.000, 57.222, 2.

## 9.3 Consolidate observations

### 9.3.1 Company groups

In [18]:
company_groups <- rename(company_groups, new_name = name)
sample <- left_join(sample, company_groups, by=c('OPERATOR_ID' = 'members'))
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
glimpse(subset(sample, !is.na(sample$new_name)))

Observations: 596
Variables: 22
Groups: OPERATOR_ID, COMMODITY [80]
$ OPERATOR_ID           <chr> "11169", "12470", "12628", "15485", "18718", "2…
$ YEAR                  <dbl> 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008,…
$ NAME                  <chr> "ENBRIDGE ENERGY, LIMITED PARTNERSHIP", "MID - …
$ COMMODITY             <chr> "crude", "crude", "crude", "crude", "crude", "c…
$ MILES                 <dbl> 1030.000, 910.000, 913.000, 55.213, 1631.219, 4…
$ AGE_UNKNOWN_MILES     <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ MILES_PRE_1940        <dbl> 0.000, 0.000, 0.000, 0.000, 371.800, 0.500, 11.…
$ MILES_1940            <dbl> 0.000, 0.000, 618.000, 0.000, 10.000, 0.000, 24…
$ MILES_1950            <dbl> 0.000, 0.000, 230.000, 26.991, 276.300, 0.000, …
$ MILES_1960            <dbl> 2.000, 0.000, 49.000, 0.000, 183.680, 0.000, 8.…
$ MILES_1970            <dbl> 4.000, 0.000, 58.000, 26.934, 138.000, 57.500, …
$ MILES_1980            <dbl> 1.000, 0.000, 39.000, 0.000, 46.0

In [19]:
nrow(sample[is.na(sample$MILES), ])

[1] 9

In [20]:
sample_no_group <- subset(sample, is.na(sample$new_name))
sample_group <- subset(sample, !is.na(sample$new_name))

sample_group <- sample_group %>%
    group_by(new_name, YEAR, COMMODITY) %>%
    summarize(OPERATOR_ID = first(new_name), 
              NAME              = first(new_name),
              MILES             = sum(MILES, na.rm=TRUE), 
              AGE_UNKNOWN_MILES = sum(AGE_UNKNOWN_MILES, na.rm=TRUE), 
              MILES_PRE_1940    = sum(MILES_PRE_1940, na.rm=TRUE), 
              MILES_1940      = sum(MILES_1940, na.rm=TRUE),
              MILES_1950      = sum(MILES_1950, na.rm=TRUE),
              MILES_1960      = sum(MILES_1960, na.rm=TRUE),
              MILES_1970      = sum(MILES_1970, na.rm=TRUE),
              MILES_1980      = sum(MILES_1980, na.rm=TRUE),
              MILES_1990      = sum(MILES_1990, na.rm=TRUE),
              MILES_2000      = sum(MILES_2000, na.rm=TRUE),
              MILES_2010      = sum(MILES_2010, na.rm=TRUE),
              PARENT            = first(PARENT), 
              INCIDENTS         = sum(INCIDENTS, na.rm=TRUE), 
              SIGNIFICANT_INCIDENTS = sum(SIGNIFICANT_INCIDENTS, na.rm=TRUE), 
              GROUP = 'group') %>%
    # We calculate the avg age and change once all groupings (including M&As below), are taken care of
    ungroup()

glimpse(sample_group)

Observations: 351
Variables: 20
$ new_name              <chr> "BP (Group)", "BP (Group)", "BP (Group)", "BP (…
$ YEAR                  <dbl> 2008, 2009, 2010, 2010, 2011, 2011, 2012, 2012,…
$ COMMODITY             <chr> "crude", "crude", "hvl", "non-hvl", "hvl", "non…
$ OPERATOR_ID           <chr> "BP (Group)", "BP (Group)", "BP (Group)", "BP (…
$ NAME                  <chr> "BP (Group)", "BP (Group)", "BP (Group)", "BP (…
$ MILES                 <dbl> 441.000, 476.000, 334.000, 587.000, 108.000, 55…
$ AGE_UNKNOWN_MILES     <dbl> 0.000, 0.000, 0.000, 100.000, 0.000, 100.000, 0…
$ MILES_PRE_1940        <dbl> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.…
$ MILES_1940            <dbl> 280.000, 48.000, 0.000, 12.000, 0.000, 12.000, …
$ MILES_1950            <dbl> 106.000, 106.000, 0.000, 152.000, 0.000, 143.00…
$ MILES_1960            <dbl> 27.000, 27.000, 130.000, 43.000, 0.000, 17.000,…
$ MILES_1970            <dbl> 7.000, 7.000, 147.000, 634.000, 22.000, 616.000…
$ MILES_1980        

In [21]:
sample_no_group$GROUP = "not group"

sample <- bind_rows(sample_group, sample_no_group)
sample <- sample[ , (names(sample) != "new_name")]
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
glimpse(sample_n(sample, 10))

Observations: 10
Variables: 22
$ YEAR                  <dbl> 2008, 2016, 2009, 2013, 2014, 2010, 2011, 2018,…
$ COMMODITY             <chr> "crude", "crude", "crude", "hvl", "non-hvl", "h…
$ OPERATOR_ID           <chr> "18201", "Phillips 66 (Group)", "30658", "Exxon…
$ NAME                  <chr> "SENECA RESOURCES CORP", "Phillips 66 (Group)",…
$ MILES                 <dbl> 8.200, 1027.660, 13.810, 1697.700, 751.269, 151…
$ AGE_UNKNOWN_MILES     <dbl> NA, 47.460, NA, 42.000, 0.000, 0.000, 0.000, 1.…
$ MILES_PRE_1940        <dbl> 0.00, 2.67, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 0.000, 481.480, 8.060, 19.000, 59.791, 0.000, 0…
$ MILES_1950            <dbl> 0.000, 877.130, 0.000, 326.000, 2.595, 188.000,…
$ MILES_1960            <dbl> 0.000, 594.690, 0.000, 562.500, 3.758, 0.000, 0…
$ MILES_1970            <dbl> 0.000, 61.590, 0.000, 396.000, 301.825, 0.000, …
$ MILES_1980            <dbl> 0.000, 48.450, 2.810, 216.000, 0.004, 0.000, 0.…
$ MILES_1990         

In [22]:
table(sample$GROUP)


    group not group 
      351       725 

### 9.3.2 M&As

In [23]:
m_as <- rename(m_as_orig, new_name = name)

# We need to fill in the NA values for start and end_year with values that always match, because when logical conditions encounter NAs
# they will always be wrong. The filtering in those cases happens because the new_name column is NA.
m_as[is.na(m_as$start_year), ]$start_year <- -9999
m_as[is.na(m_as$end_year), ]$end_year <- 9999

years = data.frame(year = unique(sample$YEAR))
m_as <- merge(m_as, years)
m_as <- m_as %>%
    filter(year >= start_year & year < end_year)

glimpse(m_as)

Observations: 20
Variables: 5
$ members    <chr> "Kinder Morgan (Group)", "31720", "Kinder Morgan (Group)",…
$ new_name   <chr> "Kinder Morgan (Group)", "Kinder Morgan (Group)", "Kinder …
$ start_year <chr> "-9999", "-9999", "-9999", "-9999", "-9999", "-9999", "-99…
$ end_year   <chr> "2013", "2013", "2013", "2013", "2013", "2013", "2013", "2…
$ year       <dbl> 2008, 2008, 2009, 2009, 2010, 2010, 2011, 2011, 2012, 2012…


In [24]:
m_as <- select(m_as, members, new_name, year)
m_as['m_a'] <- TRUE

sample <- left_join(sample, m_as, by=c('OPERATOR_ID' = 'members', 'YEAR' = 'year'))
sample$m_a <- replace_na(sample$m_a, FALSE)

glimpse(sample)

Observations: 1,076
Variables: 24
$ YEAR                  <dbl> 2008, 2009, 2010, 2010, 2011, 2011, 2012, 2012,…
$ COMMODITY             <chr> "crude", "crude", "hvl", "non-hvl", "hvl", "non…
$ OPERATOR_ID           <chr> "BP (Group)", "BP (Group)", "BP (Group)", "BP (…
$ NAME                  <chr> "BP (Group)", "BP (Group)", "BP (Group)", "BP (…
$ MILES                 <dbl> 441.000, 476.000, 334.000, 587.000, 108.000, 55…
$ AGE_UNKNOWN_MILES     <dbl> 0.000, 0.000, 0.000, 100.000, 0.000, 100.000, 0…
$ MILES_PRE_1940        <dbl> 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.…
$ MILES_1940            <dbl> 280.000, 48.000, 0.000, 12.000, 0.000, 12.000, …
$ MILES_1950            <dbl> 106.000, 106.000, 0.000, 152.000, 0.000, 143.00…
$ MILES_1960            <dbl> 27.000, 27.000, 130.000, 43.000, 0.000, 17.000,…
$ MILES_1970            <dbl> 7.000, 7.000, 147.000, 634.000, 22.000, 616.000…
$ MILES_1980            <dbl> 5.000, 4.000, 33.000, 14.000, 5.000, 10.000, 12…
$ MILES_1990      

In [25]:
sample_m_as <- subset(sample, m_a == TRUE)
sample_no_m_as <- subset(sample, m_a == FALSE)

glimpse(sample_m_as)

Observations: 34
Variables: 24
$ YEAR                  <dbl> 2017, 2017, 2017, 2018, 2018, 2018, 2017, 2017,…
$ COMMODITY             <chr> "crude", "hvl", "non-hvl", "crude", "hvl", "non…
$ OPERATOR_ID           <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ NAME                  <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ MILES                 <dbl> 2998.00, 170.00, 874.70, 2759.44, 163.74, 854.4…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 47.76, 240.…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 9.00, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00, 239.1…
$ MILES_1950            <dbl> 1.00, 188.00, 134.00, 1.00, 188.82, 133.81, 0.0…
$ MILES_1960            <dbl> 20.00, 0.00, 273.30, 19.36, 0.00, 273.30, 0.00,…
$ MILES_1970            <dbl> 7.00, 0.00, 85.50, 7.39, 0.00, 85.50, 0.13, 260…
$ MILES_1980            <dbl> 0.00, 0.00, 1.60, 0.00, 0.00, 1.80, 26.10, 47.9…
$ MILES_1990         

In [26]:
sample_m_as <- sample_m_as %>%
    group_by(new_name, YEAR, COMMODITY) %>%
    summarize(OPERATOR_ID = first(new_name), 
              NAME              = first(new_name),
              MILES             = sum(MILES, na.rm=TRUE), 
              AGE_UNKNOWN_MILES = sum(AGE_UNKNOWN_MILES, na.rm=TRUE), 
              MILES_PRE_1940    = sum(MILES_PRE_1940, na.rm=TRUE), 
              MILES_1940        = sum(MILES_1940, na.rm=TRUE),
              MILES_1950        = sum(MILES_1950, na.rm=TRUE),
              MILES_1960        = sum(MILES_1960, na.rm=TRUE),
              MILES_1970        = sum(MILES_1970, na.rm=TRUE),
              MILES_1980        = sum(MILES_1980, na.rm=TRUE),
              MILES_1990        = sum(MILES_1990, na.rm=TRUE),
              MILES_2000        = sum(MILES_2000, na.rm=TRUE),
              MILES_2010        = sum(MILES_2010, na.rm=TRUE),
              PARENT            = first(PARENT), 
              INCIDENTS         = sum(INCIDENTS, na.rm=TRUE), 
              SIGNIFICANT_INCIDENTS = sum(SIGNIFICANT_INCIDENTS, na.rm=TRUE), 
              GROUP = 'm&a') %>%
    ungroup()

glimpse(sample_m_as)

Observations: 26
Variables: 20
$ new_name              <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ YEAR                  <dbl> 2017, 2017, 2017, 2018, 2018, 2018, 2008, 2009,…
$ COMMODITY             <chr> "crude", "hvl", "non-hvl", "crude", "hvl", "non…
$ OPERATOR_ID           <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ NAME                  <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ MILES                 <dbl> 3407.00, 170.00, 874.70, 3201.98, 163.74, 854.4…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 9.00, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1950            <dbl> 247.00, 188.00, 134.00, 247.62, 188.82, 133.81,…
$ MILES_1960            <dbl> 20.00, 0.00, 273.30, 19.75, 0.00, 273.30, 0.00,…
$ MILES_1970            <dbl> 7.00, 0.00, 85.50, 7.39, 0.00, 85.50, 0.00, 0.0…
$ MILES_1980         

In [27]:
sample <- bind_rows(sample_m_as, sample_no_m_as)
sample <- sample[ , !(names(sample) %in% c("new_name", "m_a"))]
testit::assert(nrow(sample[duplicated(select(sample, YEAR, COMMODITY, OPERATOR_ID)), ]) == 0)
glimpse(sample)

Observations: 1,068
Variables: 22
$ YEAR                  <dbl> 2017, 2017, 2017, 2018, 2018, 2018, 2008, 2009,…
$ COMMODITY             <chr> "crude", "hvl", "non-hvl", "crude", "hvl", "non…
$ OPERATOR_ID           <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ NAME                  <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ MILES                 <dbl> 3407.00, 170.00, 874.70, 3201.98, 163.74, 854.4…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 9.00, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1950            <dbl> 247.00, 188.00, 134.00, 247.62, 188.82, 133.81,…
$ MILES_1960            <dbl> 20.00, 0.00, 273.30, 19.75, 0.00, 273.30, 0.00,…
$ MILES_1970            <dbl> 7.00, 0.00, 85.50, 7.39, 0.00, 85.50, 0.00, 0.0…
$ MILES_1980            <dbl> 0.00, 0.00, 1.60, 0.00, 0.00, 1.80, 0.00, 0.00,…
$ MILES_1990      

## 9.4 Mark M&A years

In [28]:
glimpse(m_as_orig)

Observations: 8
Variables: 4
$ members    <chr> "Sunoco (Group)", "Energy Transfer (Group)", "Kinder Morga…
$ name       <chr> "Sunoco (Group)", "Sunoco (Group)", "Kinder Morgan (Group)…
$ start_year <chr> "2017", "2017", NA, NA, "2017", "2017", "2018", "2018"
$ end_year   <chr> NA, NA, "2013", "2013", NA, NA, NA, NA


In [29]:
m_as_orig$start_year <- as.numeric(m_as_orig$start_year)
m_as_orig <- select(m_as_orig, -c(name))

m_as_start <- subset(m_as_orig, !is.na(start_year))
m_as_start$m_a_start <- TRUE

sample <- left_join(sample, m_as_start, by=c("OPERATOR_ID" = "members", "YEAR" = "start_year"))
glimpse(sample)

Observations: 1,068
Variables: 24
$ YEAR                  <dbl> 2017, 2017, 2017, 2018, 2018, 2018, 2008, 2009,…
$ COMMODITY             <chr> "crude", "hvl", "non-hvl", "crude", "hvl", "non…
$ OPERATOR_ID           <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ NAME                  <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ MILES                 <dbl> 3407.00, 170.00, 874.70, 3201.98, 163.74, 854.4…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 9.00, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1950            <dbl> 247.00, 188.00, 134.00, 247.62, 188.82, 133.81,…
$ MILES_1960            <dbl> 20.00, 0.00, 273.30, 19.75, 0.00, 273.30, 0.00,…
$ MILES_1970            <dbl> 7.00, 0.00, 85.50, 7.39, 0.00, 85.50, 0.00, 0.0…
$ MILES_1980            <dbl> 0.00, 0.00, 1.60, 0.00, 0.00, 1.80, 0.00, 0.00,…
$ MILES_1990      

In [30]:
m_as_orig$end_year <- as.numeric(m_as_orig$end_year)

m_as_end <- subset(m_as_orig, !is.na(end_year))
# We add 1 to the year, so the year we match on is the first year after the group has ceased to exist.
m_as_end$end_year <- m_as_end$end_year + 1
m_as_end$m_a_end <- TRUE

sample <- left_join(sample, m_as_end, by=c("OPERATOR_ID" = "members", "YEAR" = "end_year"))
glimpse(sample)

Observations: 1,068
Variables: 26
$ YEAR                  <dbl> 2017, 2017, 2017, 2018, 2018, 2018, 2008, 2009,…
$ COMMODITY             <chr> "crude", "hvl", "non-hvl", "crude", "hvl", "non…
$ OPERATOR_ID           <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ NAME                  <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ MILES                 <dbl> 3407.00, 170.00, 874.70, 3201.98, 163.74, 854.4…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 9.00, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1950            <dbl> 247.00, 188.00, 134.00, 247.62, 188.82, 133.81,…
$ MILES_1960            <dbl> 20.00, 0.00, 273.30, 19.75, 0.00, 273.30, 0.00,…
$ MILES_1970            <dbl> 7.00, 0.00, 85.50, 7.39, 0.00, 85.50, 0.00, 0.0…
$ MILES_1980            <dbl> 0.00, 0.00, 1.60, 0.00, 0.00, 1.80, 0.00, 0.00,…
$ MILES_1990      

In [31]:
subset(sample, m_a_end == TRUE)

YEAR,COMMODITY,OPERATOR_ID,NAME,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,⋯,INCIDENTS,SIGNIFICANT_INCIDENTS,GROUP,PERC_OFFSHORE,AVG_AGE,CHANGE,end_year,m_a_start,start_year,m_a_end
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<dbl>,<lgl>
2014,crude,Kinder Morgan (Group),Kinder Morgan (Group),423.06,0,0,0.00,0.00,0.00,⋯,0,0,group,NA,NA,NA,NA,NA,NA,TRUE
2014,hvl,Kinder Morgan (Group),Kinder Morgan (Group),194.20,0,0,0.00,0.25,0.00,⋯,0,0,group,NA,NA,NA,NA,NA,NA,TRUE
2014,non-hvl,Kinder Morgan (Group),Kinder Morgan (Group),4735.19,0,0,50.63,272.96,1315.77,⋯,13,6,group,NA,NA,NA,NA,NA,NA,TRUE
2014,crude,31720,"EXPRESS HOLDINGS (USA), LLC",212.40,0,0,0.00,812.20,0.00,⋯,2,1,not group,0,49.49706,-6.9,NA,NA,NA,TRUE


In [32]:
sample$M_A <- (sample$m_a_start | sample$m_a_end)
sample <- select(sample, -c(end_year, m_a_start, start_year, m_a_end))
sample$M_A <- replace_na(sample$M_A, FALSE)

glimpse(sample)

Observations: 1,068
Variables: 23
$ YEAR                  <dbl> 2017, 2017, 2017, 2018, 2018, 2018, 2008, 2009,…
$ COMMODITY             <chr> "crude", "hvl", "non-hvl", "crude", "hvl", "non…
$ OPERATOR_ID           <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ NAME                  <chr> "Enbridge (Group)", "Enbridge (Group)", "Enbrid…
$ MILES                 <dbl> 3407.00, 170.00, 874.70, 3201.98, 163.74, 854.4…
$ AGE_UNKNOWN_MILES     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_PRE_1940        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1940            <dbl> 9.00, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00, 0.00,…
$ MILES_1950            <dbl> 247.00, 188.00, 134.00, 247.62, 188.82, 133.81,…
$ MILES_1960            <dbl> 20.00, 0.00, 273.30, 19.75, 0.00, 273.30, 0.00,…
$ MILES_1970            <dbl> 7.00, 0.00, 85.50, 7.39, 0.00, 85.50, 0.00, 0.0…
$ MILES_1980            <dbl> 0.00, 0.00, 1.60, 0.00, 0.00, 1.80, 0.00, 0.00,…
$ MILES_1990      

## 9.5 Calculate avg age, change for new groups

In [33]:
sample <- sample %>%
    group_by(OPERATOR_ID, COMMODITY) %>%
    arrange(YEAR) %>%
    mutate(CHANGE = MILES - lag(MILES, 1))
glimpse(subset(sample, YEAR > 2010))

Observations: 710
Variables: 23
Groups: OPERATOR_ID, COMMODITY [100]
$ YEAR                  <dbl> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011,…
$ COMMODITY             <chr> "crude", "hvl", "non-hvl", "hvl", "non-hvl", "c…
$ OPERATOR_ID           <chr> "Kinder Morgan (Group)", "Kinder Morgan (Group)…
$ NAME                  <chr> "Kinder Morgan (Group)", "Kinder Morgan (Group)…
$ MILES                 <dbl> 196.000, 483.490, 4601.250, 108.000, 559.000, 0…
$ AGE_UNKNOWN_MILES     <dbl> 0.000, 0.000, 0.000, 0.000, 100.000, 0.000, 0.0…
$ MILES_PRE_1940        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ MILES_1940            <dbl> 0.000, 0.000, 24.320, 0.000, 12.000, 0.000, 0.0…
$ MILES_1950            <dbl> 248.000, 0.250, 322.690, 0.000, 143.000, 0.000,…
$ MILES_1960            <dbl> 0.000, 0.000, 1499.080, 0.000, 17.000, 0.000, 4…
$ MILES_1970            <dbl> 0.000, 1202.090, 563.540, 22.000, 616.000, 0.62…
$ MILES_1980            <dbl> 0.000, 0.160, 716.940, 5.000, 10

In [34]:
sample$AVG_AGE <- ((sample$MILES_PRE_1940 * 90 +
                    sample$MILES_1940 * 75 +
                    sample$MILES_1950 * 65 +
                    sample$MILES_1960 * 55 +
                    sample$MILES_1970 * 45 +
                    sample$MILES_1980 * 35 +
                    sample$MILES_1990 * 25 +
                    sample$MILES_2000 * 15 +
                    sample$MILES_2010 * 05) /
                   (sample$MILES_PRE_1940 +
                    sample$MILES_1940 +
                    sample$MILES_1950 +
                    sample$MILES_1960 +
                    sample$MILES_1970 +
                    sample$MILES_1980 +
                    sample$MILES_1990 +
                    sample$MILES_2000 +
                    sample$MILES_2010))

glimpse(sample)

Observations: 1,068
Variables: 23
Groups: OPERATOR_ID, COMMODITY [228]
$ YEAR                  <dbl> 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008,…
$ COMMODITY             <chr> "crude", "crude", "crude", "crude", "crude", "c…
$ OPERATOR_ID           <chr> "Kinder Morgan (Group)", "BP (Group)", "Enbridg…
$ NAME                  <chr> "Kinder Morgan (Group)", "BP (Group)", "Enbridg…
$ MILES                 <dbl> 190.000, 441.000, 1497.000, 385.080, 1951.000, …
$ AGE_UNKNOWN_MILES     <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, NA, NA, NA, NA…
$ MILES_PRE_1940        <dbl> 0.000, 0.000, 0.000, 270.033, 96.000, 0.500, 0.…
$ MILES_1940            <dbl> 0.000, 280.000, 444.000, 166.000, 686.000, 0.00…
$ MILES_1950            <dbl> 243.000, 106.000, 1.000, 325.000, 882.000, 0.00…
$ MILES_1960            <dbl> 0.000, 27.000, 3.000, 50.000, 432.000, 0.000, 1…
$ MILES_1970            <dbl> 0.000, 7.000, 7.000, 29.000, 434.000, 57.500, 2…
$ MILES_1980            <dbl> 0.000, 5.000, 2.000, 72.981, 2

## 9.6 Save

In [35]:
feather::write_feather(sample, paste0("../preprocessed_data/sample_consolidated_", Sys.Date(), ".feather"))
haven::write_dta(sample, paste0("../stata_data/sample_consolidated_", Sys.Date(), ".dta"))